In [8]:
# Importamos las librerías.
from bs4 import BeautifulSoup
import requests
import csv

En esta parte del código, iteramos sobre una lista de URLs que corresponden a diferentes secciones dentro de la categoría "**Tecnología**" de MercadoLibre.

In [9]:
URLs_BASE = ['https://www.mercadolibre.com.ar/c/celulares-y-telefonos#menu=categories', 
             'https://www.mercadolibre.com.ar/c/computacion#menu=categories',
             'https://www.mercadolibre.com.ar/c/camaras-y-accesorios#menu=categories',
             'https://www.mercadolibre.com.ar/c/electronica-audio-y-video#menu=categories']

reseñas_individuales = []

for url in URLs_BASE:
    pedido_obtenido = requests.get(url)
    
    if pedido_obtenido.status_code == 200: # Nos aseguramos que la página se descargó bien.
        soup = BeautifulSoup(pedido_obtenido.text, "html.parser")

        # Buscamos contenedores de productos.
        enlaces_productos = soup.find_all('a', class_ = 'splinter-link')

        for enlace in enlaces_productos:
            url_producto = enlace.get('href')
            
            # Filtramos solo las URLs que contienen "/p/"
            if url_producto and "/p/" in url_producto:
                producto_pagina = requests.get(url_producto)

                if producto_pagina.status_code == 200: # Nos aseguramos que la página se descargó bien.
                    producto_soup = BeautifulSoup(producto_pagina.text, "html.parser")

                    # Extraemos la información del producto.
                    nombre_producto = producto_soup.find('h1', class_ = 'ui-pdp-title')
                    precio_producto = producto_soup.find('span', class_ = 'andes-money-amount__fraction')
                    rating_producto = producto_soup.find('span', class_ = 'ui-pdp-review__rating')
                    reseña_producto = producto_soup.find_all('p', class_ = 'ui-review-capability-comments__comment__content ui-review-capability-comments__comment__content')

                    # Convertimos los datos a texto.
                    nombre_producto = nombre_producto.text.strip() if nombre_producto else "Sin nombre"
                    precio_producto = precio_producto.text.strip() if precio_producto else "Sin precio"
                    rating_producto = rating_producto.text.strip() if rating_producto else "Sin rating"

                    # Para las reseñas, extraemos todas en una lista.
                    for reseña in reseña_producto:
                        texto_reseña = reseña.text.strip()
                        reseñas_individuales.append([nombre_producto, precio_producto, rating_producto, texto_reseña])

    else:
        print(f'Error al acceder a {url}')

In [12]:
# Guardamos en un archivo CSV
with open('reseñas_mercadolibre.csv', 'w', newline = '', encoding = 'utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(['Nombre del producto', 'Precio', 'Rating', 'Reseña'])
    writer.writerows(reseñas_individuales)